In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'n2o': 3.2e-07, 'ch4': 1.8e-06}
band = [9]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0]
  ng_refs = [7]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(2, 1.6, 1.6, 1.6, 1.6, 1.6, 1.8)]
  wgt = [(0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.9)]
o n2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 3.2e-07
  dv = 0.001
  klin = 2.22e-20
  molecule = n2o
  ng_adju = [0, 0]
  ng_refs = [2, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(1.8, 1.66), (1.6, 1.8)]
  wgt = [(0.6, 0.5), (0.6, 0.9)]
o ch4 band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 1.8e-06
 

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.066328                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.005961
0.000750      2.0 -0.060461                  2.0 -0.002462
0.001052      3.0 -0.047631                  3.0  0.005639
0.001476      4.0 -0.033380                  4.0  0.014487
0.002070      5.0 -0.017776                  5.0  0.024226
0.002904      6.0 -0.000730                  6.0  0.034950
0.004074      7.0  0.017819                  7.0  0.046720
0.005714      8.0  0.037991                  8.0  0.059623
0.008015      9.0  0.059865                  9.0  0.073744
0.011243     10.0  0.083531                 10.0  0.089203
0.015771     11.0  0.109058                 11.0  0.106075
0.022122     12.0  0.136502                 12.0  0.124464
0.031031     13.0  0.165792                 13.0  0.144391
0.043528     14.0  0.192936                 14.0  0.163412
0.061057     15.0  0.213790                 15.0  0.179000
0.085645     16.0  0.231293                 16.0  0.193079
0.120136     17.0  0.248479                 17.0  0.207746
0.168516     18.0  0.265355                 18.0  0.223029
0.236378     19.0  0.281956                 19.0  0.238912
0.331549     20.0  0.298677                 20.0  0.255354
0.465100     21.0  0.313966                 21.0  0.270564
0.652400     22.0  0.287321                 22.0  0.256623
0.915100     23.0  0.212346                 23.0  0.207184
1.283650     24.0  0.140092                 24.0  0.154513
1.800600     25.0  0.087034                 25.0  0.109971
2.525700     26.0  0.050197                 26.0  0.072935
3.542800     27.0  0.025894                 27.0  0.043109
4.969550     28.0  0.013512                 28.0  0.022890
6.970850     29.0  0.010056                 29.0  0.012445
9.778100     30.0  0.008907                 30.0  0.007209
13.715850    31.0  0.007261                 31.0  0.004137
19.239350    32.0  0.005019                 32.0  0.002211
26.987250    33.0  0.004521                 33.0  0.002618
37.855300    34.0  0.005973                 34.0  0.004753
53.100050    35.0  0.006915                 35.0  0.006117
73.887500    36.0  0.007257                 36.0  0.006562
97.662500    37.0  0.006990                 37.0  0.006077
121.437500   38.0  0.005345                 38.0  0.004161
145.212500   39.0  0.003023                 39.0  0.001493
168.987500   40.0  0.000702                 40.0 -0.001372
192.762500   41.0 -0.001800                 41.0 -0.004509
216.537500   42.0 -0.004669                 42.0 -0.008029
240.312500   43.0 -0.008044                 43.0 -0.011944
264.087500   44.0 -0.012244                 44.0 -0.016392
287.862500   45.0 -0.017333                 45.0 -0.021119
311.637500   46.0 -0.019732                 46.0 -0.023109
335.412500   47.0 -0.016985                 47.0 -0.020433
359.187500   48.0 -0.012151                 48.0 -0.015575
382.962500   49.0 -0.006934                 49.0 -0.010116
406.737500   50.0 -0.001017                 50.0 -0.003936
430.512500   51.0  0.005343                 51.0  0.002777
454.287500   52.0  0.012053                 52.0  0.009633
478.062500   53.0  0.019007                 53.0  0.016539
501.837500   54.0  0.026127                 54.0  0.023786
525.612500   55.0  0.033322                 55.0  0.031542
549.387500   56.0  0.040446                 56.0  0.039337
573.162500   57.0  0.047340                 57.0  0.046586
596.937500   58.0  0.053781                 58.0  0.053038
620.712500   59.0  0.059557                 59.0  0.058740
644.487500   60.0  0.064495                 60.0  0.063989
668.262500   61.0  0.068514                 61.0  0.068686
692.037500   62.0  0.071656                 62.0  0.072941
715.812500   63.0  0.074078                 63.0  0.076516
739.587500   64.0  0.076069                 64.0  0.079520
76

# Fluxes by Layer

CRD                        CLIRAD                \
                        flug      fldg     fnetg      flug          fldg   
pressure     level                                                         
0.000000e+00 1     -6.887330  0.000000 -6.887330       NaN           NaN   
1.000000e-08 1           NaN       NaN       NaN -6.854793  9.420961e-08   
6.244000e-04 2     -6.887338  0.000004 -6.887335 -6.854796  2.579643e-06   
8.759000e-04 3     -6.887342  0.000006 -6.887336 -6.854797  3.641079e-06   
1.228600e-03 4     -6.887346  0.000008 -6.887338 -6.854799  5.302632e-06   
1.723400e-03 5     -6.887353  0.000012 -6.887340 -6.854801  7.894361e-06   
2.417400e-03 6     -6.887360  0.000019 -6.887342 -6.854803  1.193283e-05   
3.390900e-03 7     -6.887370  0.000028 -6.887342 -6.854805  1.822152e-05   
4.756500e-03 8     -6.887383  0.000044 -6.887339 -6.854807  2.800340e-05   
6.672000e-03 9     -6.887398  0.000067 -6.887330 -6.854809  4.320014e-05   
9.358900e-03 10    -6.887415  0.000104 -6.887311 -6.854809  6.678133e-05   
1.312780e-02 11    -6.887435  0.000161 -6.887274 -6.854806  1.033320e-04   
1.841450e-02 12    -6.887454  0.000248 -6.887206 -6.854796  1.599107e-04   
2.583020e-02 13    -6.887468  0.000382 -6.887086 -6.854774  2.473689e-04   
3.623230e-02 14    -6.887468  0.000587 -6.886882 -6.854731  3.823060e-04   
5.082340e-02 15    -6.887444  0.000896 -6.886548 -6.854654  5.878823e-04   
7.129060e-02 16    -6.887380  0.001350 -6.886030 -6.854527  8.947113e-04   
1.000000e-01 17    -6.887254  0.002011 -6.885243 -6.854323  1.348008e-03   
1.402710e-01 18    -6.887021  0.002963 -6.884058 -6.854001  2.016536e-03   
1.967600e-01 19    -6.886610  0.004328 -6.882282 -6.853492  3.000415e-03   
2.759970e-01 20    -6.885901  0.006265 -6.879636 -6.852693  4.444237e-03   
3.871000e-01 21    -6.884693  0.008988 -6.875705 -6.851442  6.553820e-03   
5.431000e-01 22    -6.882651  0.012748 -6.869903 -6.849495  9.607745e-03   
7.617000e-01 23    -6.879770  0.017307 -6.862463 -6.846851  1.360964e-02   
1.068500e+00 24    -6.876573  0.021828 -6.854746 -6.843871  1.816004e-02   
1.498800e+00 25    -6.873366  0.025761 -6.847605 -6.840782  2.294752e-02   
2.102400e+00 26    -6.870251  0.028869 -6.841382 -6.837693  2.772171e-02   
2.949000e+00 27    -6.867379  0.031031 -6.836348 -6.834772  3.211597e-02   
4.136600e+00 28    -6.865026  0.032321 -6.832705 -6.832286  3.569509e-02   
5.802500e+00 29    -6.863343  0.033305 -6.830039 -6.830386  3.831324e-02   
8.139200e+00 30    -6.862173  0.034918 -6.827255 -6.829000  4.037195e-02   
1.141700e+01 31    -6.861585  0.037788 -6.823797 -6.828289  4.246043e-02   
1.601470e+01 32    -6.862187  0.042345 -6.819842 -6.828972  4.539709e-02   
2.246400e+01 33    -6.865325  0.049317 -6.816008 -6.832420  5.053395e-02   
3.151050e+01 34    -6.872055  0.060892 -6.811163 -6.839782  6.070241e-02   
4.420010e+01 35    -6.882631  0.080446 -6.802185 -6.851707  7.977149e-02   
6.200000e+01 36    -6.898493  0.110889 -6.787605 -6.870313  1.112775e-01   
8.577500e+01 37    -6.921013  0.153846 -6.767167 -6.897915  1.573622e-01   
1.095500e+02 38    -6.945043  0.197564 -6.747480 -6.928641  2.052043e-01   
1.333250e+02 39    -6.972105  0.239679 -6.732425 -6.963978  2.522606e-01   
1.571000e+02 40    -7.004073  0.280161 -6.723912 -7.005558  2.980461e-01   
1.808750e+02 41    -7.041552  0.319618 -6.721934 -7.053876  3.425008e-01   
2.046500e+02 42    -7.085324  0.358319 -6.727005 -7.109629  3.855549e-01   
2.284250e+02 43    -7.136296  0.396141 -6.740155 -7.173652  4.269620e-01   
2.522000e+02 44    -7.195940  0.433132 -6.762809 -7.247087  4.667544e-01   
2.759750e+02 45    -7.266746  0.469455 -6.797291 -7.331652  5.051518e-01   
2.997500e+02 46    -7.352177  0.506072 -6.846106 -7.429045  5.430610e-01   
3.235250e+02 47    -7.450765  0.549087 -6.901678 -7.536651  5.855780e-01   
3.473000e+02 48    -7.555745  0.606231 -6.949514 -7.648406  6.397820e-01   
3.710750e+02 49    -7.662868  0.679134 -6.983734 -7.760677  7.081843e-01 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')